## Problem Statement
### 1) Data cleaning  in Python / R. <br>2) Summary table in R/ Python which shows the important characteristics like count for each categorical variable.<br>3) Data visualization in Tableau.

In [2]:
#!pip install fancyimpute
#!pip install bqplot
#!pip install ipywidgets
#!pip install ipympl
#!jupyter labextension install @jupyter-widgets/jupyterlab-manager
#!jupyter nbextension install --user --py widgetsnbextension
#!jupyter nbextension enable --user --py widgetsnbextension
#!pip install plotly
#!pip install cufflinks
import requests
import csv
import pandas as pd
from pandas import DataFrame as df
import re

import os
from pandas.io.json import json_normalize
import urllib.request as request
import json

import operator
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import ipywidgets as widgets
#import plotly.express as px
#import plotly.graph_objects as go
#import bqplot
#from bqplot import Pie
#from collections import Counter
#from bqplot import DateScale, LinearScale, Lines, Axis, Figure
#from bqplot import Tooltip
#import cufflinks as cf
from matplotlib.patches import ConnectionPatch
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt

from IPython.core.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))


from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
pd.set_option('display.max_columns', 50)
%matplotlib widget


In [3]:
dataset= pd.read_csv("MENTAL HEALTH DATASET.csv")

In [4]:
dataset.describe()

,Age
count,1.259000e+03
mean,7.942815e+07
std,2.818299e+09
min,-1.726000e+03
25%,2.700000e+01
50%,3.100000e+01
75%,3.600000e+01
max,1.000000e+11


## Checking Null Vaues


In [5]:
dataset.isnull().values.any()


True

#### Columns with Null values

In [6]:
print("Columns with number of null values ")
for i in dataset.columns.values:
    #print(i, "is null values : ",dataset[i].isnull().values.any())
    if dataset[i].isnull().values.any():
        print(i, ": " ,dataset[i].isnull().sum())

Columns with number of null values 
state :  515
self_employed :  18
work_interfere :  264
comments :  1095


#### Non-Null Values

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   state                      744 non-null    object
 5   self_employed              1241 non-null   object
 6   family_history             1259 non-null   object
 7   treatment                  1259 non-null   object
 8   work_interfere             995 non-null    object
 9   no_employees               1259 non-null   object
 10  remote_work                1259 non-null   object
 11  tech_company               1259 non-null   object
 12  benefits                   1259 non-null   object
 13  care_options               1259 non-null   object
 14  wellness

In [8]:
dataset = dataset.drop(['comments'], axis= 1)
dataset = dataset.drop(['state'], axis= 1)
#dataset = dataset.drop(['work_interfere'], axis= 1)

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1259 entries, 0 to 1258
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1259 non-null   object
 1   Age                        1259 non-null   int64 
 2   Gender                     1259 non-null   object
 3   Country                    1259 non-null   object
 4   self_employed              1241 non-null   object
 5   family_history             1259 non-null   object
 6   treatment                  1259 non-null   object
 7   work_interfere             995 non-null    object
 8   no_employees               1259 non-null   object
 9   remote_work                1259 non-null   object
 10  tech_company               1259 non-null   object
 11  benefits                   1259 non-null   object
 12  care_options               1259 non-null   object
 13  wellness_program           1259 non-null   object
 14  seek_hel

# Data Cleaning


In [10]:
Int_def = 0
String_def = 'NaN'
Float_def = 0.0

# Create lists by data tpe
intColumns = ['Age']
stringColumns = ['Gender', 'Timestamp', 'Country', 'self_employed', 'family_history', 'treatment', 'work_interfere',
                 'no_employees', 'remote_work', 'tech_company', 'anonymity', 'leave', 'mental_health_consequence',
                 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
                 'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 'wellness_program',
                 'seek_help']
floatColumns = []

# Clean the NaN's
for column in dataset:
    if column in intColumns:
        dataset[column] = dataset[column].fillna(Int_def)
    elif column in stringColumns:
        dataset[column] = dataset[column].fillna(String_def)
    elif column in floatColumns:
        dataset[column] = dataset[column].fillna(Float_def)
    else:
        print('Error: Feature %s not recognized.' % column)
dataset.head(5)  

,Timestamp,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
0,2014-08-27 11:29:31,37,Female,United States,NaN,No,Yes,Often,6-25,No,Yes,Yes,Not sure,No,Yes,Yes,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No
1,2014-08-27 11:29:37,44,M,United States,NaN,No,No,Rarely,More than 1000,No,No,Don't know,No,Don't know,Don't know,Don't know,Don't know,Maybe,No,No,No,No,No,Don't know,No
2,2014-08-27 11:29:44,32,Male,Canada,NaN,No,No,Rarely,6-25,No,Yes,No,No,No,No,Don't know,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,Yes,Yes,Often,26-100,No,Yes,No,Yes,No,No,No,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes
4,2014-08-27 11:30:22,31,Male,United States,NaN,No,No,Never,100-500,Yes,Yes,Yes,No,Don't know,Don't know,Don't know,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No


### Column = Gender

In [11]:
gender = dataset['Gender'].str.lower()

#Select unique elements
gender = dataset['Gender'].unique()

#Made gender groups
male_ = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_ = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female_ = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for (row, col) in dataset.iterrows():

    if str.lower(col.Gender) in male_:
        dataset['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_:
        dataset['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_:
        dataset['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

list_ = ['A little about you', 'p']
dataset = dataset[~dataset['Gender'].isin(list_)]
print(dataset['Gender'].unique())

['female' 'male' 'trans']


### Column = self_employed

In [12]:
print("%self employed : ",dataset[dataset.self_employed == "Yes"].shape[0]/dataset.shape[0]*100)
print("%not self employed : ",dataset[dataset.self_employed == "No"].shape[0]/dataset.shape[0]*100)


%self employed :  11.455847255369928
%not self employed :  87.11217183770883


### As NaN values are very less in number and %of self employed are very less we can replace the NaN values with Not Self Employed categry

In [13]:
dataset['self_employed'] = dataset['self_employed'].replace([String_def], 'No')
print(dataset['self_employed'].unique())

['No' 'Yes']


In [14]:
print("%often work interference : ",dataset[dataset.work_interfere == "Often"].shape[0]/dataset.shape[0]*100)
print("%rarely work interference : ",dataset[dataset.work_interfere == "Rarely"].shape[0]/dataset.shape[0]*100)
print("%never work interference : ",dataset[dataset.work_interfere == "Never"].shape[0]/dataset.shape[0]*100)
print("%sometimes work interference : ",dataset[dataset.work_interfere == "Sometimes"].shape[0]/dataset.shape[0]*100)

%often work interference :  11.296738265712012
%rarely work interference :  13.762927605409706
%never work interference :  16.94510739856802
%sometimes work interference :  36.9928400954654


In [15]:
#interference and not interference % are almost same so let up replace it by not known
dataset['work_interfere'] = dataset['work_interfere'].replace([String_def], 'not known' )
print(dataset['work_interfere'].unique())

['Often' 'Rarely' 'Never' 'Sometimes' 'not known']


## Column = Age

In [16]:
dataset[dataset["Age"]<0]

,Timestamp,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
143,2014-08-27 12:39:14,-29,male,United States,No,No,No,not known,More than 1000,Yes,No,Yes,No,Don't know,Yes,Don't know,Don't know,No,No,Some of them,Yes,No,No,Don't know,No
715,2014-08-28 10:07:53,-1726,male,United Kingdom,No,No,Yes,Sometimes,26-100,No,No,No,No,No,No,Don't know,Somewhat difficult,Yes,No,No,No,No,Maybe,Don't know,No


In [17]:
dataset = dataset.drop([143, 715])

In [19]:
dataset[dataset["Age"]>80]

,Timestamp,Age,Gender,Country,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,care_options,wellness_program,seek_help,anonymity,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence
364,2014-08-27 15:05:21,329,male,United States,No,No,Yes,Often,6-25,Yes,Yes,Yes,Yes,No,No,Don't know,Don't know,Maybe,No,Some of them,No,No,No,No,No
390,2014-08-27 15:24:47,99999999999,trans,Zimbabwe,Yes,Yes,Yes,Often,1-5,No,Yes,No,Yes,No,No,No,Very difficult,Yes,Yes,No,No,Yes,No,No,Yes


In [20]:
dataset = dataset.drop([364, 390])

In [22]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1253 entries, 0 to 1258
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Timestamp                  1253 non-null   object
 1   Age                        1253 non-null   int64 
 2   Gender                     1253 non-null   object
 3   Country                    1253 non-null   object
 4   self_employed              1253 non-null   object
 5   family_history             1253 non-null   object
 6   treatment                  1253 non-null   object
 7   work_interfere             1253 non-null   object
 8   no_employees               1253 non-null   object
 9   remote_work                1253 non-null   object
 10  tech_company               1253 non-null   object
 11  benefits                   1253 non-null   object
 12  care_options               1253 non-null   object
 13  wellness_program           1253 non-null   object
 14  seek_hel

In [23]:
dataset.to_csv("mental_health_cleaned_data.csv")